In [1]:
from math import ceil
import random

# Pasos de bebe, pasos de gigante

In [2]:
def paso_bebe_paso_gigante(g, y, n):
    m = ceil(pow(n, 1/2))
    g %= n + 1
    y %= n + 1

    tabla_hash = [(pow(g, j, n + 1), j) for j in range(m)]
    beta = pow(pow(g, -1, n + 1), m, n + 1)
    gamma = y
    
    for i in range(m):
        if any(a == gamma for a, _ in tabla_hash):
            x = i * m + next(b for a, b in tabla_hash if a == gamma)
            print(f'm = {m}\nix = {i}\njx = {next(b for a, b in tabla_hash if a == gamma)}\nx = {x}')
            break
        gamma = gamma * beta % (n + 1)
    else:
        x = None

    return x

In [3]:
p = 3129553951
g = 19
y = 1132858435
n = p - 1

paso_bebe_paso_gigante(g, y, n)

m = 55943
ix = 5725
jx = 1327
x = 320275002


320275002

# Pollard's Rho

In [4]:
def MCD(x, y):
    if (x < y):
        b = x
        a = y
    else:
        a = x
        b = y
    while (b > 0):
        r = a % b
        a = b
        b = r
    return a

In [5]:
def obtener_valor_binario(alpha):
    # Convierte alpha a su representación binaria como cadena de caracteres
    binario = bin(alpha)
    # Extrae el valor binario eliminando el prefijo '0b' y lo convierte a entero
    return int(binario[2:])

def fs(alpha, m):
    b_alpha = obtener_valor_binario(alpha)
    return b_alpha % m

In [6]:
def almacenar_tabla(g, y, n, m):
    R = []
    g %= n + 1
    y %= n + 1

    for _ in range(1, m):
        uj = random.randint(0, n-1)
        vj = random.randint(0, n-1)
        aj = (g**uj * y**vj) % n
        R.append((aj, uj, vj))

    return R

In [7]:
def caminata(xab, R, m, n):
    x, a, b = xab
    j = fs(x, m)
    if (j == 0):
        return (pow(x, 2, n), (2 * a) % n, (2 * b) % n)
    else:
        aj, uj, vj = R[j-1]
        return (aj * x % n, (a + uj) % n, (b + vj) % n)

In [9]:
def pollard(g, y, n, m):
    R = almacenar_tabla(g, y, n, m)
    a0 = random.randint(0, n-1); b0 = random.randint(0, n-1); x0 = pow(g, a0) * pow(y, b0)

    (xi, ai, bi) = caminata((x0, a0, b0), R, m, n)
    (x2i, a2i, b2i) = caminata(caminata((x0, a0, b0), R, m, n), R, m, n)

    while (xi != x2i) :
        (xi, ai, bi) = caminata((xi, ai, bi), R, m, n)
        (x2i, a2i, b2i) = caminata(caminata((x2i, a2i, b2i), R, m, n), R, m, n)

    if (MCD(bi - b2i, n) == 1):
        return pow((a2i - ai) * (bi - b2i), -1, n)
    else:
        return 'Fallo'

In [24]:
g = 2
y = 3
n = 17
m = 5

pollard(g, y, n, m)

12